In [1]:
import os
import re

#import fitz  # requires fitz, PyMuPDF
import pdfrw
import subprocess
import os.path
import sys

In [19]:
#https://westhealth.github.io/exploring-fillable-forms-with-pdfrw.html

In [4]:
#template_out = pdfrw.PdfReader('./daca/pdf_temp/test_marked.pdf')

In [14]:
#template.keys()

In [15]:
#len(template.pages)
# there are 7 pages

## 1. Test with the first page

## Print out all the columns and description

In [2]:
ANNOT_KEY = '/Annots'           # key for all annotations within a page
ANNOT_FIELD_KEY = '/T'          # Name of field. i.e. given ID of field
ANNOT_FORM_type = '/FT'         # Form type (e.g. text/button)
ANNOT_FORM_button = '/Btn'      # ID for buttons, i.e. a checkbox
ANNOT_FORM_text = '/Tx'         # ID for textbox
SUBTYPE_KEY = '/Subtype'
WIDGET_SUBTYPE_KEY = '/Widget'
PARENT_KEY = '/Parent'

In [3]:
data_dict = {
    "form1[0].#subform[0].G28_Attached[0]": True,
    "form1[0].#subform[0].#area[0].AttyLicenseNum[0]": "test number",
    "form1[0].#subform[0].P1_Line1_2a_Checkbox[0]": True,
    "form1[0].#subform[0].P1_Line1_2a_Checkbox[1]": True,
    "form1[0].#subform[0].P1_Line4c_Unit[0]": True,
    "form1[0].#subform[0].P1_Line4c_Unit[1]": True,
    "form1[0].#subform[0].P1_Line4c_Unit[2]": True
}

In [4]:
barcode_list = ['form1[0].#subform[0].PDF417BarCode1[0]',
                'form1[0].#subform[1].PDF417BarCode1[1]',
                'form1[0].#subform[2].PDF417BarCode1[2]',
                'form1[0].#subform[3].PDF417BarCode1[3]',
                'form1[0].#subform[4].PDF417BarCode1[4]',
                'form1[0].#subform[5].PDF417BarCode1[5]',
                'form1[0].#subform[6].PDF417BarCode1[6]']

In [5]:
template_pdf = pdfrw.PdfReader('./daca/i-821d_test-unlocked.pdf')

In [6]:
def text_form(annotation, value):
    annotation.update(pdfrw.PdfDict(V='{}'.format(value)))
    annotation.update(pdfrw.PdfDict(AP=''))
    #pdfstr = pdfrw.objects.pdfstring.PdfString.encode(value)
    #annotation.update(pdfrw.PdfDict(V=pdfstr, AS=pdfstr))

In [14]:
def checkbox(annotation, key_value, export=None):
    if export:
        export = '/' + export
    else:
        # determine the value to trigger the checkbox
        keys = annotation['/AP']['/N'].keys()
        if '/Off' in keys:
            keys.remove('/Off')
        export = keys[0]
    if key_value:
        annotation.update(pdfrw.PdfDict(V=export, AS=export))
    else:
        del annotation['/V']
        del annotation['/AS']

In [26]:
def combobox(annotation, value):
    export = None
    for each in annotation['/Opt']:
        if each[1].to_unicode() == value:
            export = each[0].to_unicode()
    if export is None: 
        raise KeyError(f"Export Value: {value} Not Found")
        
    pdfstr = pdfrw.objects.pdfstring.PdfString.encode(export)
    annotation.update(pdfrw.PdfDict(V=pdfstr, AS=pdfstr))
        

In [15]:
page1 = template_pdf.pages[0]
annotations = page1['/Annots']

In [16]:
annotation = annotations[24]

In [22]:
export = None
for each in annotation['/Opt']:
    if each[1].to_unicode() == 'CA':
        export = each[0].to_unicode()
        print(export)
if export is None:
    raise KeyError(f"Export Value: CA Not Found")

CA


In [29]:
page1 = template_pdf.pages[0]
annotations = page1['/Annots']

form_type = []
for page in template_pdf.pages:
    annotations = page['/Annots']
    for annotation in annotations: 
        ft = annotation['/FT']
        ff = annotation['/Ff']
        #print(annotation['/T'])
    
        if ft == '/Tx':
            form_type.append("text")
        if ft == '/Ch':
            if ff and int(ff) & 1 << 17:
                form_type.append("combo")
            else: 
                form_type.append("list")
        if ft == '/Btn':
            if ff and int(ff) & 1 << 15:
                form_type.append("radio")
            else:
                form_type.append("checkbox")
                

In [32]:
set(form_type)

{'checkbox', 'combo', 'text'}

In [77]:
for page in template_pdf.pages:
    annotations = page[ANNOT_KEY]
    if annotations is None:
        continue

    for annotation in annotations:
        if annotation[SUBTYPE_KEY]==WIDGET_SUBTYPE_KEY:
            if not annotation[ANNOT_FIELD_KEY]:
                annotation=annotation[PARENT_KEY]
            if annotation[ANNOT_FIELD_KEY]:
                key = annotation[ANNOT_FIELD_KEY].to_unicode()
                if key in barcode_list:
                    continue 
                    
                if key in data_dict.keys():
                    print("Key: ", key)
                    print("data_dict_value: ", data_dict[key])
                    if annotation[ANNOT_FORM_type] == ANNOT_FORM_button: # fill checkbox
                        print(annotation['/AP']['/N'].keys())
                        checkbox(annotation, data_dict[key], export=None)
                    
                    if annotation[ANNOT_FORM_type] == ANNOT_FORM_text: # fill text
                        text_form(annotation, data_dict[key])

[WARNING] tokens.py:221 Did not find PDF object (0, 0) (line=2530, col=1, token='endobj')


Key:  form1[0].#subform[0].G28_Attached[0]
data_dict_value:  True
['/Y', '/Off']
Key:  form1[0].#subform[0].#area[0].AttyLicenseNum[0]
data_dict_value:  test number
Key:  form1[0].#subform[0].P1_Line1_2a_Checkbox[0]
data_dict_value:  True
['/Off', '/I']
Key:  form1[0].#subform[0].P1_Line1_2a_Checkbox[1]
data_dict_value:  True
['/R']
Key:  form1[0].#subform[0].P1_Line4c_Unit[0]
data_dict_value:  True
['/Off', '/ FLR ']
Key:  form1[0].#subform[0].P1_Line4c_Unit[1]
data_dict_value:  True
['/ STE ', '/Off']
Key:  form1[0].#subform[0].P1_Line4c_Unit[2]
data_dict_value:  True
['/ APT ', '/Off']


In [78]:
template_pdf.Root.AcroForm.update(pdfrw.PdfDict(NeedAppearances=pdfrw.PdfObject('true')))
pdfrw.PdfWriter().write('./daca/pdf_temp/test_marked.pdf', template_pdf)

In [46]:
data_dict = {
    #"form1[0].#subform[0].G28_Attached[0]": "",
    #"form1[0].#subform[0].#area[0].AttyLicenseNum[0]": "",
    "form1[0].#subform[0].P1_Line3a_Name[0]": "Fang",
    "form1[0].#subform[0].P1_Line3b_Name[0]": "Jiaqi",
    #"form1[0].#subform[0].P1_Line3c_Name[0]": "",
    "form1[0].#subform[0].P1_Line2b_Date[0]": "01/31/2022",
    "form1[0].#subform[0].P1_Line1_2a_Checkbox[0]": False, 
    "form1[0].#subform[0].P1_Line1_2a_Checkbox[1]": True,
    #"form1[0].#subform[0].P1_Line4a_Name[0]": "",
    "form1[0].#subform[0].P1_Line4b_Street[0]": "39967 Waxwing Drive",
    #"form1[0].#subform[0].P1_Line4c_Number[0]": "",
    #"form1[0].#subform[0].P1_Line4c_Unit[0]":"",
    #"form1[0].#subform[0].P1_Line4c_Unit[1]": "",
    #"form1[0].#subform[0].P1_Line4c_Unit[2]": "",
    "form1[0].#subform[0].P1_Line5_Checkbox[0]": False,
    "form1[0].#subform[0].P1_Line5_Checkbox[1]": True,
    "form1[0].#subform[0].P1_Line5a_Checkbox[0]": False, 
    "form1[0].#subform[0].P1_Line5b_Checkbox[0]": False,
    "form1[0].#subform[0].P1_Line5c_Checkbox[0]": False, 
    "form1[0].#subform[0].P1_Line5d_Checkbox[0]": False,
    #"form1[0].#subform[0].P1_Line5g_Location[0]": "",
    #"form1[0].#subform[0].P1_Line5f_Date[0]": "",
    "form1[0].#subform[0].P1_Line5e_Checkbox[0]": False,
    "form1[0].#subform[0].P1_Line4f_ZipCode[0]": '94560',
    "form1[0].#subform[0].P1_Line4e_State[0]": 'CA',
    "form1[0].#subform[0].P1_Line4d_City[0]": 'Newark'
    #"form1[0].#subform[0].PDF417BarCode1[0]":""
}

In [26]:
annotations[0]

{'/Border': ['0', '0', '0'],
 '/Rect': ['253.736', '570.973', '263.736', '580.973'],
 '/T': '(form1[0].#subform[0].G28_Attached[0])',
 '/F': '4',
 '/BS': {'/W': '0'},
 '/Subtype': '/Widget',
 '/DA': '(/ZapfDingbatsITC 8 Tf 0 g)',
 '/MK': {'/CA': '(8)'},
 '/TU': '(To Be Completed by an Attorney or Accredited Representative, if any. Select this box if Form G-28 is attached to represent the requestor.)',
 '/AP': {'/N': {'/Y': {'/Filter': '/FlateDecode',
    '/Type': '/XObject',
    '/Subtype': '/Form',
    '/FormType': '1',
    '/BBox': ['0', '0', '10', '10'],
    '/Resources': {'/ProcSet': ['/PDF']},
    '/Length': '63'},
   '/Off': {'/Filter': '/FlateDecode',
    '/Type': '/XObject',
    '/Subtype': '/Form',
    '/FormType': '1',
    '/BBox': ['0', '0', '0', '0'],
    '/Resources': {'/ProcSet': ['/PDF']},
    '/Length': '11'}},
  '/D': {'/Y': {'/Filter': '/FlateDecode',
    '/Type': '/XObject',
    '/Subtype': '/Form',
    '/FormType': '1',
    '/BBox': ['0', '0', '10', '10'],
    '/Res